# Orchestrating Jobs, Model Registration, Continuous Deployment, and Lineage Tracking with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Workflow service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

And the SageMaker Lineage service makes it easy to track all the artifacts created in a SageMaker Machine Learning Pipeline from start to finish.

# Use Case: Fine-Tune a BERT Model and Create a Text Classifier

We have chosen the [Amazon Customer Reviews Dataset](https://s3.amazonaws.com/amazon-reviews-pds/readme.html) as our main dataset.

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.

![BERT Training](img/bert_training.png)

BERT’s attention mechanism is called a Transformer. This is, not coincidentally, the name of the popular BERT Python library, “Transformers,” maintained by a company called [HuggingFace](https://github.com/huggingface/transformers). We will use a variant of BERT called [DistilBert](https://arxiv.org/pdf/1910.01108.pdf) which requires less memory and compute, but maintains very good accuracy on our dataset.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Parametrized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.


## Our BERT Pipeline

In the Processing Step, we perform Feature Engineering to create BERT embeddings from the `reviews_body` text using the pre-trained BERT model, and split the dataset into train, validation and test files. To optimize for Tensorflow training, we saved the files in TFRecord format. 

In the Training Step, we fine-tune the BERT model to our Customer Reviews Dataset and add a new classification layer to predict the `star_rating` for a given `review_body`.

In the Evaluation Step, we take the trained model and a test dataset as input, and produce a JSON file containing classification evaluation metrics.

In the Condition Step, we decide whether to register this model if the accuracy of the model, as determined by our evaluation step exceeded some value. 



![](./img/bert_sagemaker_pipeline.png)

## SageMaker Model Registry

Amazon SageMaker Model Registry supports the following:

* Catalog models after the training step - data scientists run tens to thousands of experiments and may select a small set of models as candidates for production.
* Manage model versions - data scientists can register new models which will be automatically versioned in the model registry.
* Compare models - data scientists can run model evaluation steps in the pipeline and generate model metrics (e.g. accuracy metrics and bias metrics) which are recorded in the Model Registry and can be used to compare model versions.
* Approve models - data scientists can mark model versions as “approved” or “rejected”. Alternately, the pipeline can also automate the model approvals. If there is a deployment pipeline associated with a Model and a live endpoint, then the model version is propagated in production. 
* Deploy models in different AWS accounts - models in the Model Registry support resource sharing across accounts which enables models built in data scientist accounts to be deployed in different pre-production and production accounts.

## SageMaker Lineage

Amazon SageMaker Lineage supports the following:

* Automatically tracks all the artifacts created in a machine learning workflow from start to finish.  Modeled as a directed graph like structure.
* Explore the lineage artifacts with easy to use SDK methods.


## Notebook Overview

This notebook shows how to:

### SageMaker Workflows

* Define a set of Workflow Parameters that can be used to parametrize a Workflow Pipeline
* Define a Processing step that performs cleaning and feature engineering, splitting the input data into train and test data sets
* Define a Training step that trains a model on the pre-processed train data set
* Define a Processing step that evaluates the trained model's performance on the test data set
* Define a Register Model step that creates a model package from the estimator and model artifacts used in training
* Define a Conditional step that measures a condition based on output from prior steps and conditionally executes the Register Model step
* Define and create a Pipeline in a Workflow DAG, with the defined parameters and steps defined
* Start a Pipeline execution and wait for execution to complete

### SageMaker Model Registry

* Create a SageMaker Project based on the Model Package Group name from the pipeline execution defined before


### SageMaker Lineage

Amazon SageMaker Lineage supports the following:

* Provide the inputs and outputs of SageMaker job artifacts

# A SageMaker Workflow

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and model registration:

![A typical ML Application pipeline](img/pipeline-full.png)

### Create SageMaker Clients and Session

First, we create a new SageMaker Session in the current AWS region. We also acquire the role arn for the session.

This role arn should be the execution role arn that you set up in the Prerequisites section of this notebook.

In [1]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Track Pipeline In `Experiment`

In [2]:
import time
timestamp = int(time.time())

pipeline_name = 'BERT-pipeline-{}'.format(timestamp)

In [3]:
import time
from smexperiments.experiment import Experiment

experiment = Experiment.create(
                experiment_name=pipeline_name,
                description='Amazon Customer Reviews BERT Pipeline Experiment', 
                sagemaker_boto_client=sm)

experiment_name = experiment.experiment_name
print('Experiment name: {}'.format(experiment_name))

ModuleNotFoundError: No module named 'smexperiments'

# Create the `Trial`

In [ ]:
import time
from smexperiments.trial import Trial

trial = Trial.create(trial_name='trial-{}'.format(timestamp),
                     experiment_name=experiment_name,
                     sagemaker_boto_client=sm)

trial_name = trial.trial_name
print('Trial name: {}'.format(trial_name))

# Create the `Experiment Config`'s for Each Step

In [ ]:
experiment_config_prepare = {
    'ExperimentName': experiment_name,
    'TrialName': trial_name,
    'TrialComponentDisplayName': 'prepare'
}

In [ ]:
experiment_config_train = {
    'ExperimentName': experiment_name,
    'TrialName': trial_name,
    'TrialComponentDisplayName': 'train'
}

In [ ]:
experiment_config_evaluate = {
    'ExperimentName': experiment_name,
    'TrialName': trial_name,
    'TrialComponentDisplayName': 'evaluate'
}

In [ ]:
experiment_config_model_register = {
    'ExperimentName': experiment_name,
    'TrialName': trial_name,
    'TrialComponentDisplayName': 'model_register'
}

# Specify the Raw Inputs S3 Location

In [ ]:
raw_input_data_s3_uri = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(raw_input_data_s3_uri)

In [ ]:
!aws s3 ls $raw_input_data_s3_uri

# Define Parameters to Parametrize Pipeline Execution

We define Workflow Parameters by which we can parametrize our Pipeline and vary the values injected and used in Pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:

* `ParameterString` - representing a `str` Python type
* `ParameterInteger` - representing an `int` Python type
* `ParameterFloat` - representing a `float` Python type

These parameters support providing a default value, which can be overridden on pipeline execution. The default value specified should be an instance of the type of the parameter.

The parameters defined in this workflow below include:

* `processing_instance_type` - The `ml.*` instance type of the processing job.
* `processing_instance_count` - The instance count of the processing job. For illustrative purposes only: 1 is the only value that makes sense here.
* `training_instance_type` - The `ml.*` instance type of the training job.
* `model_approval_status` - What approval status to register the trained model with for CI/CD purposes. Defaults to "PendingManualApproval". (NOTE: not available in service yet)
* `input_data` - The URL location of the input data

# Pipeline Parameters

In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

# Experiment Parameters

In [ ]:
exp_name = ParameterString(
    name="ExperimentName",
    default_value=experiment_name,
)

# Processing Step Parameters

In [ ]:
input_data = ParameterString(
    name="InputData",
    default_value=raw_input_data_s3_uri,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)

processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.c5.2xlarge"
)

In [ ]:
max_seq_length = ParameterInteger(
    name="MaxSeqLength",
    default_value=64,
)

balance_dataset = ParameterString(
    name="BalanceDataset",
    default_value="True",
)
    
train_split_percentage = ParameterFloat(
    name="TrainSplitPercentage",
    default_value=0.90,
)

validation_split_percentage = ParameterFloat(
    name="ValidationSplitPercentage",
    default_value=0.05,
)

test_split_percentage = ParameterFloat(
    name="TestSplitPercentage",
    default_value=0.05,
)

feature_store_offline_prefix = ParameterString(
    name="FeatureStoreOfflinePrefix",
    default_value="reviews-feature-store-" + str(timestamp),
)

feature_group_name = ParameterString(
    name="FeatureGroupName",
    default_value="reviews-feature-group-" + str(timestamp)
)

In [ ]:
!pygmentize ./preprocess-scikit-text-to-bert-feature-store.py

We create an instance of an `SKLearnProcessor` processor and we use that in our `ProcessingStep`.

We also specify the `framework_version` we will use throughout.

Note the `processing_instance_type` and `processing_instance_count` parameters that used by the processor instance.

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(framework_version='0.23-1',
                             role=role,
                             instance_type=processing_instance_type,
                             instance_count=processing_instance_count,
                             env={'AWS_DEFAULT_REGION': region},                             
                             # max_runtime_in_seconds=7200
                            )

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processing_inputs=[
        ProcessingInput(
            input_name='raw-input-data',
            source=input_data,
            destination='/opt/ml/processing/input/data/',
            s3_data_distribution_type='ShardedByS3Key'
        )
]

processing_outputs=[
        ProcessingOutput(output_name='bert-train',
                         s3_upload_mode='EndOfJob',
                         source='/opt/ml/processing/output/bert/train',
                        ),
        ProcessingOutput(output_name='bert-validation',
                         s3_upload_mode='EndOfJob',                         
                         source='/opt/ml/processing/output/bert/validation',
                        ),
        ProcessingOutput(output_name='bert-test',
                         s3_upload_mode='EndOfJob',
                         source='/opt/ml/processing/output/bert/test',
                        ),
]        

processing_step = ProcessingStep(
    name='Processing', 
    code='preprocess-scikit-text-to-bert-feature-store.py',
    processor=processor,
    inputs=processing_inputs,
    outputs=processing_outputs,
    job_arguments=['--train-split-percentage', str(train_split_percentage.default_value),                   
                   '--validation-split-percentage', str(validation_split_percentage.default_value),
                   '--test-split-percentage', str(test_split_percentage.default_value),
                   '--max-seq-length', str(max_seq_length.default_value),
                   '--balance-dataset', str(balance_dataset.default_value),
                   '--feature-store-offline-prefix', str(feature_store_offline_prefix.default_value),
                   '--feature-group-name', str(feature_group_name.default_value)
                  ]
)        

print(processing_step)

![Define a Processing Step for Feature Engineering](img/pipeline-2.png)

Finally, we use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to a processor instance's `run` method, for those familiar with the existing Python SDK.

Note the `input_data` parameters passed into `ProcessingStep` as the input data of the step itself. This input data will be used by the processor instance when it is run.

Also, take note the `"bert-train"`, `"bert-validation"` and `"bert-test"` named channels specified in the output configuration for the processing job. Such step `Properties` can be used in subsequent steps and will resolve to their runtime values at execution. In particular, we'll call out this usage when we define our training step.

# Train Step

In [ ]:
train_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.c5.9xlarge"
)

train_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=1
)

# Setup Training Hyper-Parameters
Note that `max_seq_length` is re-used from the processing hyper-parameters above

In [ ]:
epochs = ParameterInteger(
    name="Epochs",
    default_value=1
)
    
learning_rate = ParameterFloat(
    name="LearningRate",
    default_value=0.00001
) 
    
epsilon = ParameterFloat(
    name="Epsilon",
    default_value=0.00000001
)
        
train_batch_size = ParameterInteger(
    name="TrainBatchSize",
    default_value=128
)
    
validation_batch_size = ParameterInteger(
    name="ValidationBatchSize",
    default_value=128
)
    
test_batch_size = ParameterInteger(
    name="TestBatchSize",
    default_value=128
)
    
train_steps_per_epoch = ParameterInteger(
    name="TrainStepsPerEpoch",
    default_value=50
)
    
validation_steps = ParameterInteger(
    name="ValidationSteps",
    default_value=50
)
    
test_steps = ParameterInteger(
    name="TestSteps",
    default_value=50
)
    
train_volume_size = ParameterInteger(
    name="TrainVolumeSize",
    default_value=1024
) 
    
use_xla = ParameterString(
    name="UseXLA",
    default_value="True",
)

use_amp = ParameterString(
    name="UseAMP",
    default_value="True",
)
    
freeze_bert_layer = ParameterString(
    name="FreezeBERTLayer",
    default_value="False",
)

enable_sagemaker_debugger = ParameterString(
    name="EnableSageMakerDebugger",
    default_value="False",
)
    
enable_checkpointing = ParameterString(
    name="EnableCheckpointing",
    default_value="False",
)

enable_tensorboard = ParameterString(
    name="EnableTensorboard",
    default_value="False",
)
    
input_mode = ParameterString(
    name="InputMode",
    default_value="File",
)

run_validation = ParameterString(
    name="RunValidation",
    default_value="True",
)

run_test = ParameterString(
    name="RunTest",
    default_value="False",
)
    
run_sample_predictions = ParameterString(
    name="RunSamplePredictions",
    default_value="False",
)

# Setup Metrics To Track Model Performance

In [ ]:
metrics_definitions = [
     {'Name': 'train:loss', 'Regex': 'loss: ([0-9\\.]+)'},
     {'Name': 'train:accuracy', 'Regex': 'accuracy: ([0-9\\.]+)'},
     {'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9\\.]+)'},
     {'Name': 'validation:accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'},
]

In [ ]:
!pygmentize src/tf_bert_reviews.py

# Define a Training Step to Train a Model

We configure an Estimator and the input dataset. A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later.

We also specify the model path where the models from training will be saved.

Note the `train_instance_type` parameter passed may be also used and passed into other places in the pipeline. In this case, the `train_instance_type` is passed into the estimator.

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='tf_bert_reviews.py',
                       source_dir='src',
                       role=role,
                       instance_count=train_instance_count, # Make sure you have at least this number of input files or the ShardedByS3Key distibution strategy will fail the job due to no data available
                       instance_type=train_instance_type,
                       volume_size=train_volume_size,                       
                       py_version='py37',
                       framework_version='2.3.1',
                       hyperparameters={'epochs': epochs,
                                        'learning_rate': learning_rate,
                                        'epsilon': epsilon,
                                        'train_batch_size': train_batch_size,
                                        'validation_batch_size': validation_batch_size,
                                        'test_batch_size': test_batch_size,                                             
                                        'train_steps_per_epoch': train_steps_per_epoch,
                                        'validation_steps': validation_steps,
                                        'test_steps': test_steps,
                                        'use_xla': use_xla,
                                        'use_amp': use_amp,                                             
                                        'max_seq_length': max_seq_length,
                                        'freeze_bert_layer': freeze_bert_layer,
                                        'enable_sagemaker_debugger': enable_sagemaker_debugger,
                                        'enable_checkpointing': enable_checkpointing,
                                        'enable_tensorboard': enable_tensorboard,                                        
                                        'run_validation': run_validation,
                                        'run_test': run_test,
                                        'run_sample_predictions': run_sample_predictions},
                       input_mode=input_mode,
                       metric_definitions=metrics_definitions,
#                       max_run=7200 # max 2 hours * 60 minutes seconds per hour * 60 seconds per minute
                      )

Finally, we use the estimator instance to construct a `TrainingStep` as well as the `Properties` of the prior `ProcessingStep` used as input in the `TrainingStep` inputs and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to an estimator's `fit` method, for those familiar with the existing Python SDK.

In particular, we pass in the `S3Uri` of the `"train"`, `"validation"` and `"test"` output channel to the `TrainingStep`. The `properties` attribute of a Workflow step match the object model of the corresponding response of a describe call. These properties can be referenced as placeholder values and are resolved, or filled in, at runtime. For example, the `ProcessingStep` `properties` attribute matches the object model of the [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response object.

In [ ]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

training_step = TrainingStep(
    name='Train',
    estimator=estimator,
    inputs={
        'train': TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                'bert-train'
            ].S3Output.S3Uri,
            content_type='text/csv'
        ),
        'validation': TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                'bert-validation'
            ].S3Output.S3Uri,
            content_type='text/csv'
        ),
        'test': TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                'bert-test'
            ].S3Output.S3Uri,
            content_type='text/csv'
        )        
    },
)

print(training_step)

![Define a Training Step to Train a Model](img/pipeline-3.png)

# Evaluation Step

First, we develop an evaluation script that will be specified in a Processing step that will perform the model evaluation.

The evaluation script `evaluation.py` takes the trained model and the test dataset as input, and produces a JSON file containing classification evaluation metrics such as accuracy.

After pipeline execution, we will examine the resulting `evaluation.json` for analysis.

The evaluation script:

* loads in the model
* reads in the test data
* issues a bunch of predictions against the test data
* builds a classification report, including accuracy
* saves the evaluation report to the evaluation directory

Next, we create an instance of a `ScriptProcessor` processor and we use that in our `ProcessingStep`.

Note the `processing_instance_type` parameter passed into the processor.

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

evaluation_processor = SKLearnProcessor(framework_version='0.23-1',
                                        role=role,
                                        instance_type=processing_instance_type,
                                        instance_count=processing_instance_count,
                                        env={'AWS_DEFAULT_REGION': region},
                                        max_runtime_in_seconds=7200)

In [ ]:
!pygmentize evaluate_model_metrics.py


We use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to a processor instance's `run` method, for those familiar with the existing Python SDK.

The `TrainingStep` and `ProcessingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) and  [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects, respectively.

In [ ]:
from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name='EvaluationReport',
    output_name='metrics',
    path='evaluation.json'
)

In [ ]:
evaluation_step = ProcessingStep(
    name='EvaluateModel',
    processor=evaluation_processor,
    code='evaluate_model_metrics.py',
    inputs=[
        ProcessingInput(
            source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination='/opt/ml/processing/input/model'
        ),
        ProcessingInput(
            source=processing_step.properties.ProcessingInputs['raw-input-data'].S3Input.S3Uri,
            destination='/opt/ml/processing/input/data'
        )
    ],
    outputs=[
        ProcessingOutput(output_name='metrics', 
                         s3_upload_mode='EndOfJob',
                         source='/opt/ml/processing/output/metrics/'),
    ],
    job_arguments=[
                   '--max-seq-length', str(max_seq_length.default_value),
                  ],
    property_files=[evaluation_report],
)

![Define a Model Evaluation Step to Evaluate the Trained Model](img/pipeline-4.png)

In [ ]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics 

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            evaluation_step.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json"
    )
)

print(model_metrics)

# Register Model Step

![](img/pipeline-5.png)

We use the estimator instance that was used for the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a Model Package. A Model Package is a reusable model artifacts abstraction that packages all ingredients necessary for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A Model Package Group is a collection of Model Packages. You can create a Model Package Group for a specific ML business problem, and you can keep adding versions/model packages into it. Typically, we expect customers to create a ModelPackageGroup for a SageMaker Workflow Pipeline so that they can keep adding versions/model packages to the group for every Workflow Pipeline run.

The construction of `RegisterModel` is very similar to an estimator instance's `register` method, for those familiar with the existing Python SDK.

In particular, we pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

Of note, we provided a specific model package group name which we will use in the Model Registry and CI/CD work later on.

In [ ]:
model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

deploy_instance_type = ParameterString(
    name="DeployInstanceType",
    default_value="ml.m5.4xlarge"
)

deploy_instance_count = ParameterInteger(
    name="DeployInstanceCount",
    default_value=1
)

In [ ]:
model_package_group_name = f"BERT-Reviews-{timestamp}"

print(model_package_group_name)

In [ ]:
inference_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.3.1",
    py_version="py37",
    instance_type=deploy_instance_type,
    image_scope="inference"
)
print(inference_image_uri)

In [ ]:
from sagemaker.workflow.step_collections import RegisterModel

register_step = RegisterModel(
    name="RegisterModel",
#    entry_point='inference.py', # Adds a Repack Step:  https://github.com/aws/sagemaker-python-sdk/blob/01c6ee3a9ec1831e935e86df58cf70bc92ed1bbe/src/sagemaker/workflow/_utils.py#L44
#    source_dir='src',
    estimator=estimator,
    image_uri=inference_image_uri, # we have to specify, by default it's using training image
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=[deploy_instance_type],
    transform_instances=["ml.c5.18xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics
)

# Create Model for Deployment Step

![](img/pipeline-5.png)


In [ ]:
from sagemaker.model import Model

model_name = 'bert-model-{}'.format(timestamp)

model = Model(
    name=model_name,
    image_uri=inference_image_uri,
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sess,
    role=role,
)

In [ ]:
from sagemaker.inputs import CreateModelInput

create_inputs = CreateModelInput(
    instance_type=deploy_instance_type, # "ml.m5.4xlarge",
)

In [ ]:
from sagemaker.workflow.steps import CreateModelStep

create_step = CreateModelStep(
    name="CreateModel",
    model=model,
    inputs=create_inputs,
)

# Define a Condition Step to Check Accuracy and Conditionally Register Model

![](img/pipeline-6.png)

Finally, we'd like to only register this model if the accuracy of the model, as determined by our evaluation step `step_eval`, exceeded some value. A `ConditionStep` allows for pipelines to support conditional execution in the pipeline DAG based on conditions of step properties. 

Below, we:

* define a `ConditionGreaterThan` on the accuracy value found in the output of the evaluation step, `step_eval`.
* use the condition in the list of conditions in a `ConditionStep`
* pass the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`

In [ ]:
min_accuracy_value = ParameterFloat(
    name="MinAccuracyValue",
    default_value=0.01
)

In [ ]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)

minimum_accuracy_condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step=evaluation_step,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value",
    ),
    right=min_accuracy_value # accuracy
)

minimum_accuracy_condition_step = ConditionStep(
    name="AccuracyCondition",
    conditions=[minimum_accuracy_condition],
    if_steps=[register_step, create_step], # success, continue with model registration
    else_steps=[], # fail, end the pipeline
)

# Define a Pipeline of Parameters, Steps, and Conditions

Let's tie it all up into a workflow pipeline so we can execute it, and even schedule it.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair so we tack on the timestamp to the name.

Note:

* All the parameters used in the definitions must be present.
* Steps passed into the pipeline need not be in the order of execution. The SageMaker Workflow service will resolve the _data dependency_ DAG as steps the execution complete.
* Steps must be unique to either pipeline step list or a single condition step if/else list.

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        input_data,
        processing_instance_count,
        processing_instance_type,
        max_seq_length,
        balance_dataset,
        train_split_percentage,
        validation_split_percentage,
        test_split_percentage,
        feature_store_offline_prefix,
        feature_group_name,
        train_instance_type,
        train_instance_count,
        epochs,
        learning_rate,
        epsilon,
        train_batch_size,
        validation_batch_size,
        test_batch_size,
        train_steps_per_epoch,
        validation_steps,
        test_steps,
        train_volume_size,
        use_xla,
        use_amp,
        freeze_bert_layer,
        enable_sagemaker_debugger,
        enable_checkpointing,
        enable_tensorboard,
        input_mode,
        run_validation,
        run_test,
        run_sample_predictions,
        min_accuracy_value,
        model_approval_status,
        deploy_instance_type,
        deploy_instance_count
    ],
    steps=[processing_step, training_step, evaluation_step, minimum_accuracy_condition_step],
    sagemaker_session=sess,
)

Let's examine the Json of the pipeline definition that meets the SageMaker Workflow Pipeline DSL specification.

By examining the definition, we're also confirming that the pipeline was well-defined, and that the parameters and step properties resolve correctly.

In [ ]:
import json
from pprint import pprint

definition = json.loads(pipeline.definition())

pprint(definition)

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [ ]:
print(experiment_name)

## Ignore the `WARNING` below

In [ ]:
response = pipeline.create(role_arn=role)

pipeline_arn = response["PipelineArn"]
print(pipeline_arn)

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [ ]:
execution = pipeline.start(
    parameters=dict(
        InputData=raw_input_data_s3_uri,
        ProcessingInstanceCount=1,
        ProcessingInstanceType='ml.c5.2xlarge',
        MaxSeqLength=64,
        BalanceDataset='True',
        TrainSplitPercentage=0.9,
        ValidationSplitPercentage=0.05,
        TestSplitPercentage=0.05,
        FeatureStoreOfflinePrefix='reviews-feature-store-'+str(timestamp),
        FeatureGroupName='reviews-feature-group-'+str(timestamp),
        LearningRate=0.000012,
        TrainingInstanceType='ml.c5.9xlarge',
        TrainingInstanceCount=1,
        Epochs=1,
        Epsilon=0.00000001,
        TrainBatchSize=128,
        ValidationBatchSize=128,
        TestBatchSize=128,
        TrainStepsPerEpoch=50,
        ValidationSteps=50,
        TestSteps=50,
        TrainVolumeSize=1024,
        UseXLA='True',
        UseAMP='True',
        FreezeBERTLayer='False',
        EnableSageMakerDebugger='False',
        EnableCheckpointing='False',
        EnableTensorboard='False',
        InputMode='File',
        RunValidation='True',
        RunTest='Fasle',
        RunSamplePredictions='False', 
        MinAccuracyValue=0.01,
        ModelApprovalStatus='PendingManualApproval', 
        DeployInstanceType='ml.m5.4xlarge',
        DeployInstanceCount=1 
    )
)

print(execution.arn)

### Workflow Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [ ]:
from pprint import pprint

execution_run = execution.describe()
pprint(execution_run)

## Add Execution Run as Trial to Experiments

In [ ]:
execution_run_name = execution_run['PipelineExecutionDisplayName']
print(execution_run_name)

In [ ]:
pipeline_execution_arn = execution_run['PipelineExecutionArn']
print(pipeline_execution_arn)

# List Execution Steps

In [ ]:
import time

# Giving the first step time to start up
time.sleep(30)

execution.list_steps()

# Wait for the Pipeline to Complete

# _Note: If this cell errors out with `WaiterError: Waiter PipelineExecutionComplete failed: Max attempts exceeded`, just re-run it and keep waiting._

In [ ]:
%%time

execution.wait()

# Wait for the Pipeline ^^ Above ^^ to Complete

# _Note: If this cell errors out with `WaiterError: Waiter PipelineExecutionComplete failed: Max attempts exceeded`, just re-run it and keep waiting._

We can list the execution steps to check out the status and artifacts:

In [ ]:
execution.list_steps()

# Examine the Evalution Metrics

Examine the resulting model evaluation after the pipeline completes. Download the resulting evaluation.json file from S3 and print the report.

In [ ]:
for execution_step in reversed(execution.list_steps()):
    if execution_step['StepName'] == 'EvaluateModel':
        processing_job_name=execution_step['Metadata']['ProcessingJob']['Arn'].split('/')[-1]

describe_evaluation_processing_job_response = sm.describe_processing_job(ProcessingJobName=processing_job_name)

evaluation_metrics_s3_uri = describe_evaluation_processing_job_response['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']
evaluation_metrics_s3_uri

In [ ]:
from pprint import pprint

evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
    evaluation_metrics_s3_uri
))

pprint(json.loads(evaluation_json))

# Download the Trained S3 Model

In [ ]:
training_job_arn=None

for execution_step in execution.list_steps():
    if execution_step["StepName"] == "Train":
        training_job_arn = execution_step["Metadata"]["TrainingJob"]["Arn"]
        
        print(execution_step)
        break
print(training_job_arn)
        
training_job_name = training_job_arn.split('/')[-1]
print(training_job_name)

In [ ]:
model_tar_s3_uri = sm.describe_training_job(TrainingJobName=training_job_name)['ModelArtifacts']['S3ModelArtifacts']


In [ ]:
!aws s3 cp $model_tar_s3_uri ./
    

In [ ]:
!mkdir -p ./downloaded_model 
!tar -zxvf model.tar.gz -C ./downloaded_model 


In [ ]:
!saved_model_cli show --all --dir ./downloaded_model/tensorflow/saved_model/0/


# List All Artifacts Generated By The Pipeline

In [ ]:
processing_job_name=None
training_job_name=None

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step['StepName'] == 'Processing':
        processing_job_name=execution_step['Metadata']['ProcessingJob']['Arn'].split('/')[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step['StepName'] == 'Train':
        training_job_name=execution_step['Metadata']['TrainingJob']['Arn'].split('/')[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# Finalize the Experiment Run Metadata

In [ ]:
# -aws-processing-job is the default name assigned by ProcessingJob
processing_job_tc = '{}-aws-processing-job'.format(processing_job_name)
print(processing_job_tc)

In [ ]:
response = sm.associate_trial_component(
    TrialComponentName=processing_job_tc,
    TrialName=trial_name
)

In [ ]:
# -aws-training-job is the default name assigned by TrainingJob
training_job_tc = '{}-aws-training-job'.format(training_job_name)
print(training_job_tc)

In [ ]:
response = sm.associate_trial_component(
    TrialComponentName=training_job_tc,
    TrialName=trial_name
)

# Log Additional Parameters within Trial

In [ ]:
from smexperiments import tracker

processing_job_tracker = tracker.Tracker.load(trial_component_name=processing_job_tc)

In [ ]:
processing_job_tracker.log_parameters({
    "balance_dataset": str(balance_dataset), 
})

# must save after logging
processing_job_tracker.trial_component.save()

In [ ]:
processing_job_tracker.log_parameters({
    "train_split_percentage": str(train_split_percentage), 
})

# must save after logging
processing_job_tracker.trial_component.save()

In [ ]:
processing_job_tracker.log_parameters({
    "validation_split_percentage": str(validation_split_percentage), 
})

# must save after logging
processing_job_tracker.trial_component.save()

In [ ]:
processing_job_tracker.log_parameters({
    "test_split_percentage": str(test_split_percentage), 
})

# must save after logging
processing_job_tracker.trial_component.save()

In [ ]:
processing_job_tracker.log_parameters({
    "max_seq_length": str(max_seq_length), 
})

# must save after logging
processing_job_tracker.trial_component.save()

In [ ]:
processing_job_tracker.log_parameters({
    "feature_store_offline_prefix": str(feature_store_offline_prefix), 
})

# must save after logging
processing_job_tracker.trial_component.save()

In [ ]:
processing_job_tracker.log_parameters({
    "feature_group_name": str(feature_group_name), 
})

# must save after logging
processing_job_tracker.trial_component.save()

# Analyze Experiment

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

time.sleep(30)

import pandas as pd
pd.set_option("max_colwidth", 500)

experiment_analytics = ExperimentAnalytics(
    experiment_name=experiment_name,
)

experiment_analytics_df = experiment_analytics.dataframe()
experiment_analytics_df

# Update Model Package Approval Status

The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

As part of SageMaker Pipelines, data scientists can register the model with approved/pending manual approval as part of the CI/CD workflow.

We can also approve the model using the SageMaker Studio UI or programmatically as shown below.

In [ ]:
for execution_step in execution.list_steps():
    if execution_step['StepName'] == 'RegisterModel':
        model_package_arn = execution_step['Metadata']['RegisterModel']['Arn']
        break
print(model_package_arn)

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",
)

print(model_package_update_response)

# Deploy Model

In [ ]:
for execution_step in execution.list_steps():
    print(execution_step['StepName'])
    if execution_step['StepName'] == 'CreateModel':
        model_arn = execution_step['Metadata']['Model']['Arn']
        break
print(model_arn)

model_name = model_arn.split('/')[-1]
print(model_name)

# Create Model Endpoint from Model Registry

In [ ]:
endpoint_config_name = 'bert-model-epc-{}'.format(timestamp)
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m5.4xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName': model_name,
        'VariantName':'AllTraffic'}])

In [ ]:
pipeline_endpoint_name = 'bert-model-ep-{}'.format(timestamp)
print("EndpointName={}".format(pipeline_endpoint_name))

create_endpoint_response = sm.create_endpoint(
    EndpointName=pipeline_endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, pipeline_endpoint_name)))


# _Wait Until the Endpoint is Deployed_

In [ ]:
%%time

while True:
    try: 
        waiter = sm.get_waiter('endpoint_in_service')
        print('Waiting for endpoint to be in `InService`...')
        waiter.wait(EndpointName=pipeline_endpoint_name)
        break;
    except:
        print('Waiting for endpoint...')
        time.sleep(30)
        
print('Endpoint deployed.')

# _Wait Until the Endpoint ^^ Above ^^ is Deployed_

# List All Artifacts

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step['StepName'] == 'Processing':
        processing_job_name=execution_step['Metadata']['ProcessingJob']['Arn'].split('/')[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step['StepName'] == 'Train':
        training_job_name=execution_step['Metadata']['TrainingJob']['Arn'].split('/')[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)


# Test the Deployed Model

In [ ]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

predictor = TensorFlowPredictor(endpoint_name=pipeline_endpoint_name,
                                sagemaker_session=sess,
                                model_name='saved_model',
                                model_version=0,
                                content_type='application/jsonlines',
                                accept_type='application/jsonlines',
                                serializer=JSONLinesSerializer(),
                                deserializer=JSONLinesDeserializer())                                

# Predict the `star_rating` with Ad Hoc `review_body` Samples

In [ ]:
inputs = [
    {"features": ["This is great!"]},
    {"features": ["This is bad."]}
]

predicted_classes = predictor.predict(inputs)

for predicted_class in predicted_classes:
    print('Predicted star_rating: {}'.format(predicted_class))

In [ ]:
import csv

df_reviews = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz', 
                         delimiter='\t', 
                         quoting=csv.QUOTE_NONE,
                         compression='gzip')

df_sample_reviews = df_reviews[['review_body', 'star_rating']].sample(n=50)
df_sample_reviews = df_sample_reviews.reset_index(drop=True)
df_sample_reviews.shape

In [ ]:
df_sample_reviews.head()

In [ ]:
import pandas as pd

def predict(review_body):
    inputs = [
        {"features": [review_body]}
    ]
    predicted_classes = predictor.predict(inputs)
    return predicted_classes[0]['predicted_label']
    
df_sample_reviews['predicted_class'] = df_sample_reviews['review_body'].map(predict)
df_sample_reviews.head(5)

# Release Resources

In [ ]:
# sm.delete_endpoint(
#      EndpointName=pipeline_endpoint_name
# )

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}